## Deploy the winning model - RF

In [14]:
import pickle                                           # save the models to a file
with open (r"SMOTE Train only RF.pickle", "rb") as input_file:          # load XGB from file to memory.
        RF = pickle.load(input_file)

In [15]:
import pandas as pd                                     # load pandas, numpy, seaborn and matplotlib for this project.
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

deploy = pd.read_csv('DataScience deploy.csv')          # Read the data from the file.
deploy.head()                                           # see the headers of the data

,Week,Month,Day,Hour,Stoch_m15,Stoch_h1,Stoch_h4,BBand_m15,BBand_h1,BBand_h4,Want
0,39,9,1,0,82.58,72.25,8.48,90.85,30.56,37.89,False
1,39,9,1,0,56.00,72.25,8.48,18.97,30.56,37.89,False
2,39,9,1,0,20.16,72.25,8.48,75.90,30.56,37.89,False
3,39,9,1,0,6.82,72.25,8.48,49.02,30.56,37.89,False
4,39,9,1,1,16.79,71.95,8.48,22.49,52.17,37.89,False


Deploy data have problems with column names. remediating that...

In [16]:
deploy.columns = ['week','month','day','hour','stoch_m15','stoch_h1','stoch_h4','bband_m15','bband_h1','bband_h4','want']

In [17]:
deploy.head()                                           # see the headers of the data

,week,month,day,hour,stoch_m15,stoch_h1,stoch_h4,bband_m15,bband_h1,bband_h4,want
0,39,9,1,0,82.58,72.25,8.48,90.85,30.56,37.89,False
1,39,9,1,0,56.00,72.25,8.48,18.97,30.56,37.89,False
2,39,9,1,0,20.16,72.25,8.48,75.90,30.56,37.89,False
3,39,9,1,0,6.82,72.25,8.48,49.02,30.56,37.89,False
4,39,9,1,1,16.79,71.95,8.48,22.49,52.17,37.89,False


In [18]:
# change the want column to 0's and 1's

#split the want to 'true' and 'false'. save series in want_n
want_n = pd.get_dummies(deploy.want)
#print (want_n)


# concat want_n to the dataset
deploy = pd.concat([deploy,want_n], axis =1 )
deploy.merge(want_n,left_index=True, right_index=True)
deploy.drop(columns=['want'], inplace = True)
deploy.drop(columns=[False], inplace = True)
deploy.rename(columns={True: 'want'}, inplace=True)
print(deploy.head(3))

y_true = deploy.want

count0=0
count1=0

for i in y_true:
    if i == 0:
        count0 += 1
    else:
        count1 += 1

print(' this is y_true. this is the real values the came from the file ')
print("0: " + str(count0) + "\n" + "1:  " + str(count1),'\n')

   week  month  day  hour  stoch_m15  stoch_h1  stoch_h4  bband_m15  bband_h1  \
0    39      9    1     0      82.58     72.25      8.48      90.85     30.56   
1    39      9    1     0      56.00     72.25      8.48      18.97     30.56   
2    39      9    1     0      20.16     72.25      8.48      75.90     30.56   

   bband_h4  want  
0     37.89     0  
1     37.89     0  
2     37.89     0  
 this is y_true. this is the real values the came from the file 
0: 1301
1:  125 



In [19]:
deploy.drop(columns=['want'], inplace = True)           # drop the want column

In [20]:
deploy.head()                                           # see the headers of the data

,week,month,day,hour,stoch_m15,stoch_h1,stoch_h4,bband_m15,bband_h1,bband_h4
0,39,9,1,0,82.58,72.25,8.48,90.85,30.56,37.89
1,39,9,1,0,56.00,72.25,8.48,18.97,30.56,37.89
2,39,9,1,0,20.16,72.25,8.48,75.90,30.56,37.89
3,39,9,1,0,6.82,72.25,8.48,49.02,30.56,37.89
4,39,9,1,1,16.79,71.95,8.48,22.49,52.17,37.89


In [21]:
y_pred_RF = RF.predict(deploy)                          # predict the y
proba_RF =   RF.predict_proba(deploy)                   # compute the probabilies..
proba_RF = proba_RF[:,1]
print ('y_pred_RF shape :' , y_pred_RF.shape)           #resulting y_pred_RF is a 1D numpy arrays of 0s and 1s. 
print ('y_pred_RF contents : ', y_pred_RF)
print (' proba_RF contents : ', proba_RF)
print ('\n' )

y_pred_RF shape : (1426,)
y_pred_RF contents :  [0 0 0 ... 0 0 0]
 proba_RF contents :  [0.  0.2 0.  ... 0.  0.  0.1]




In [22]:
count0=0
count1=0

for i in y_pred_RF :
    if i == 0:
        count0 += 1
    else:
        count1 += 1

print(' this is the predicted y column. ')        
print("0: " + str(count0) + "\n" + "1: " + str(count1),'\n')

 this is the predicted y column. 
0: 1424
1: 2 



In [23]:
from sklearn import metrics

print ('Random Forest Scores : ' )
print ('\nConfusion Matrix' )
print (metrics.confusion_matrix(y_true=y_true, y_pred=y_pred_RF) )      # confusion matrix
print ('\nClassification Report' )
print (metrics.classification_report(y_true, y_pred_RF))                # classification report for RF
print ('-------------------------------------------------------')
print ('\n' )

Random Forest Scores : 

Confusion Matrix
[[1299    2]
 [ 125    0]]

Classification Report
             precision    recall  f1-score   support

          0       0.91      1.00      0.95      1301
          1       0.00      0.00      0.00       125

avg / total       0.83      0.91      0.87      1426

-------------------------------------------------------




In [11]:
deploy.head(4)

,week,month,day,hour,stoch_m15,stoch_h1,stoch_h4,bband_m15,bband_h1,bband_h4
0,39,9,1,0,82.58,72.25,8.48,90.85,30.56,37.89
1,39,9,1,0,56.00,72.25,8.48,18.97,30.56,37.89
2,39,9,1,0,20.16,72.25,8.48,75.90,30.56,37.89
3,39,9,1,0,6.82,72.25,8.48,49.02,30.56,37.89


In [13]:
columns = ['want']
df = pd.DataFrame(y_pred_RF,columns=columns)

df.want.head(5)

0    0
1    0
2    0
3    0
4    0
Name: want, dtype: uint8

In [65]:

final_result_for_saving_to_file = pd.concat([deploy,df.want], axis =1 )           # concat want_n to the dataset

In [68]:
deploy.head(4)

,week,month,day,hour,stoch_m15,stoch_h1,stoch_h4,bband_m15,bband_h1,bband_h4
0,39,9,1,0,82.58,72.25,8.48,90.85,30.56,37.89
1,39,9,1,0,56.00,72.25,8.48,18.97,30.56,37.89
2,39,9,1,0,20.16,72.25,8.48,75.90,30.56,37.89
3,39,9,1,0,6.82,72.25,8.48,49.02,30.56,37.89


In [67]:
final_result_for_saving_to_file.head(4)

,week,month,day,hour,stoch_m15,stoch_h1,stoch_h4,bband_m15,bband_h1,bband_h4,want
0,39,9,1,0,82.58,72.25,8.48,90.85,30.56,37.89,0
1,39,9,1,0,56.00,72.25,8.48,18.97,30.56,37.89,0
2,39,9,1,0,20.16,72.25,8.48,75.90,30.56,37.89,0
3,39,9,1,0,6.82,72.25,8.48,49.02,30.56,37.89,0


In [69]:
#final_result_for_saving_to_file.to_csv('deploy.csv')